Reference: https://www.kaggle.com/code/pavansanagapati/knowledge-graph-nlp-tutorial-bert-spacy-nltk/notebook

In [6]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [7]:
# import DOJ sentences
candidate_sentences = pd.read_csv("doj_api_data_new.csv")
candidate_sentences.shape

(48050, 2)

In [9]:
candidate_sentences['body'].sample(5)

8280     BUFFALO, N.Y.--U.S. Attorney William J. Hochul, Jr. announced today that Myron Johnson, 39, of Buffalo, N.Y., who was convicted of possession with intent to distribute cocaine, was sentenced to 51...
24831    Follow @SDILNews \n\nNAEEM MAHMOOD KOHLI, 60, of Effingham, Illinois, was convicted of seven counts of illegal dispensation of a Schedule II Controlled Substance following a 17-day jury trial held...
4303     WASHINGTON – The Department of Justice and the Department of the Interior announced today that Freeport-McMoRan Corporation and Freeport-McMoRan Morenci Inc. (Freeport-McMoRan) have agreed to pay ...
4782     A federal judge in Worcester, Mass., sentenced William Scott Dion today to 84 months in prison for conspiring to defraud the United States, and for obstructing the Internal Revenue Service (IRS), ...
33373    St. Louis, MO – JOEY D. WOOD pled  guilty to filing four false tax returns for himself and two others claiming  refunds totaling over $23,000 for tax years

### Entities Extraction
To build a knowledge graph, the most important things are the nodes and the edges between them.

In [10]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [11]:
get_entities("the film had 200 patents")

['film', '200  patents']

In [14]:
entity_pairs = []

for i in tqdm(candidate_sentences["body"]):
  entity_pairs.append(get_entities(i))

100%|███████████████████████████████████| 48050/48050 [2:02:33<00:00,  6.53it/s]


In [15]:
entity_pairs[10:20]

[['dangerous Ervin Prenci', 'criminal Fugitive Investigative More'],
 ['30 Americas participants', 'legal American States'],
 ['serious  ERO', 'illegally United country'],
 ['', ''],
 ['I', 'forward  questions'],
 ['Western office', 'crucial  David'],
 ['so cyber we', 'important  me'],
 ['also  I', 'forward  questions'],
 ['international  efforts', 'currently Northern visit'],
 ['true Justice Department', 'passing']]

### Relationship Extraction

In [27]:
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1",[pattern]) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [28]:
get_relation("John completed the task")

'completed'

In [ ]:
relations = [get_relation(i) for i in tqdm(candidate_sentences['body'])]




  0%|                                                 | 0/48050 [00:00<?, ?it/s]


  0%|                                         | 2/48050 [00:00<53:13, 15.05it/s]


  0%|                                         | 4/48050 [00:00<46:54, 17.07it/s]


  0%|                                         | 6/48050 [00:00<56:42, 14.12it/s]


  0%|                                         | 8/48050 [00:00<51:07, 15.66it/s]


  0%|                                      | 10/48050 [00:00<1:02:12, 12.87it/s]


  0%|                                        | 13/48050 [00:00<48:24, 16.54it/s]


  0%|                                      | 15/48050 [00:01<1:42:09,  7.84it/s]


  0%|                                      | 17/48050 [00:02<2:40:01,  5.00it/s]


  0%|                                      | 18/48050 [00:02<3:22:06,  3.96it/s]


  0%|                                      | 19/48050 [00:02<3:01:22,  4.41it/s]


  0%|                                      | 22/48050 [00:02<1:53:12,  7.07it/s]


 

  0%|▏                                    | 188/48050 [00:16<1:16:29, 10.43it/s]


  0%|▏                                    | 190/48050 [00:16<1:22:44,  9.64it/s]


  0%|▏                                    | 192/48050 [00:16<1:40:00,  7.98it/s]


  0%|▏                                    | 195/48050 [00:17<1:16:32, 10.42it/s]


  0%|▏                                    | 197/48050 [00:17<1:14:50, 10.66it/s]


  0%|▏                                    | 199/48050 [00:17<1:10:56, 11.24it/s]


  0%|▏                                    | 201/48050 [00:17<1:08:53, 11.58it/s]


  0%|▏                                    | 203/48050 [00:17<1:04:01, 12.45it/s]


  0%|▏                                    | 205/48050 [00:17<1:05:49, 12.12it/s]


  0%|▏                                      | 208/48050 [00:17<55:37, 14.33it/s]


  0%|▏                                      | 210/48050 [00:18<59:11, 13.47it/s]


  0%|▏                                    | 212/48050 [00:18<1:07:47, 11.76it/s]


  0%

  1%|▎                                    | 369/48050 [00:32<1:18:10, 10.17it/s]


  1%|▎                                    | 371/48050 [00:32<1:14:56, 10.60it/s]


  1%|▎                                    | 373/48050 [00:32<1:06:55, 11.87it/s]


  1%|▎                                      | 376/48050 [00:32<55:20, 14.36it/s]


  1%|▎                                    | 378/48050 [00:33<1:11:58, 11.04it/s]


  1%|▎                                    | 380/48050 [00:33<1:05:34, 12.11it/s]


  1%|▎                                    | 382/48050 [00:33<1:30:05,  8.82it/s]


  1%|▎                                    | 384/48050 [00:33<1:24:31,  9.40it/s]


  1%|▎                                    | 386/48050 [00:34<1:20:11,  9.91it/s]


  1%|▎                                    | 388/48050 [00:34<1:19:18, 10.02it/s]


  1%|▎                                    | 390/48050 [00:34<1:22:30,  9.63it/s]


  1%|▎                                    | 392/48050 [00:34<1:18:58, 10.06it/s]


  1%

  1%|▍                                    | 551/48050 [00:49<1:03:08, 12.54it/s]


  1%|▍                                    | 553/48050 [00:49<1:00:40, 13.05it/s]


  1%|▍                                      | 556/48050 [00:49<52:55, 14.96it/s]


  1%|▍                                      | 558/48050 [00:49<49:21, 16.04it/s]


  1%|▍                                    | 560/48050 [00:49<1:04:16, 12.31it/s]


  1%|▍                                    | 562/48050 [00:49<1:15:18, 10.51it/s]


  1%|▍                                    | 564/48050 [00:50<1:19:21,  9.97it/s]


  1%|▍                                    | 567/48050 [00:50<1:04:55, 12.19it/s]


  1%|▍                                    | 569/48050 [00:50<1:11:33, 11.06it/s]


  1%|▍                                    | 571/48050 [00:50<1:06:29, 11.90it/s]


  1%|▍                                    | 573/48050 [00:50<1:09:31, 11.38it/s]


  1%|▍                                    | 575/48050 [00:51<1:09:27, 11.39it/s]


  1%

  2%|▌                                    | 737/48050 [01:05<1:07:33, 11.67it/s]


  2%|▌                                    | 739/48050 [01:05<1:05:03, 12.12it/s]


  2%|▌                                    | 741/48050 [01:05<1:14:29, 10.58it/s]


  2%|▌                                    | 743/48050 [01:06<1:10:37, 11.16it/s]


  2%|▌                                    | 745/48050 [01:06<1:01:38, 12.79it/s]


  2%|▌                                    | 747/48050 [01:06<1:00:51, 12.96it/s]


  2%|▌                                    | 749/48050 [01:06<1:37:33,  8.08it/s]


  2%|▌                                    | 751/48050 [01:07<1:36:51,  8.14it/s]


  2%|▌                                    | 753/48050 [01:07<1:22:23,  9.57it/s]


  2%|▌                                    | 755/48050 [01:07<1:20:52,  9.75it/s]


  2%|▌                                    | 757/48050 [01:07<1:20:14,  9.82it/s]


  2%|▌                                    | 759/48050 [01:07<1:11:11, 11.07it/s]


  2%

...

 12%|████▏                               | 5579/48050 [08:39<1:07:39, 10.46it/s]


 12%|████▏                               | 5581/48050 [08:39<1:06:38, 10.62it/s]


 12%|████▏                               | 5583/48050 [08:39<1:02:16, 11.36it/s]


 12%|████▏                               | 5585/48050 [08:39<1:03:30, 11.14it/s]


 12%|████▍                                 | 5587/48050 [08:39<59:55, 11.81it/s]


 12%|████▍                                 | 5589/48050 [08:39<56:47, 12.46it/s]


 12%|████▍                                 | 5591/48050 [08:40<54:06, 13.08it/s]


 12%|████▍                                 | 5594/48050 [08:40<45:09, 15.67it/s]


 12%|████▍                                 | 5596/48050 [08:40<58:04, 12.18it/s]


 12%|████▍                                 | 5598/48050 [08:40<55:37, 12.72it/s]


 12%|████▏                               | 5600/48050 [08:40<1:09:44, 10.14it/s]


 12%|████▏                               | 5602/48050 [08:41<1:04:01, 11.05it/s]


 12%

 12%|████▎                               | 5762/48050 [08:56<1:00:14, 11.70it/s]


 12%|████▎                               | 5764/48050 [08:56<1:03:20, 11.13it/s]


 12%|████▎                               | 5766/48050 [08:56<1:01:58, 11.37it/s]


 12%|████▌                                 | 5768/48050 [08:56<54:02, 13.04it/s]


 12%|████▌                                 | 5770/48050 [08:57<49:26, 14.25it/s]


 12%|████▌                                 | 5773/48050 [08:57<42:22, 16.63it/s]


 12%|████▌                                 | 5775/48050 [08:57<43:31, 16.19it/s]


 12%|████▌                                 | 5777/48050 [08:57<49:19, 14.29it/s]


 12%|████▌                                 | 5779/48050 [08:57<52:22, 13.45it/s]


 12%|████▌                                 | 5781/48050 [08:57<48:30, 14.52it/s]


 12%|████▌                                 | 5783/48050 [08:57<50:52, 13.85it/s]


 12%|████▌                                 | 5785/48050 [08:58<51:58, 13.55it/s]


 12%

 12%|████▍                               | 5949/48050 [09:12<1:02:08, 11.29it/s]


 12%|████▋                                 | 5951/48050 [09:12<57:10, 12.27it/s]


 12%|████▍                               | 5953/48050 [09:12<1:02:35, 11.21it/s]


 12%|████▍                               | 5955/48050 [09:12<1:13:34,  9.53it/s]


 12%|████▍                               | 5957/48050 [09:13<1:29:35,  7.83it/s]


 12%|████▍                               | 5958/48050 [09:13<1:27:21,  8.03it/s]


 12%|████▍                               | 5959/48050 [09:13<1:25:13,  8.23it/s]


 12%|████▍                               | 5961/48050 [09:13<1:17:42,  9.03it/s]


 12%|████▍                               | 5963/48050 [09:13<1:11:26,  9.82it/s]


 12%|████▍                               | 5965/48050 [09:13<1:07:42, 10.36it/s]


 12%|████▋                                 | 5967/48050 [09:14<57:48, 12.13it/s]


 12%|████▍                               | 5969/48050 [09:14<1:14:27,  9.42it/s]


 12%

 13%|████▊                                 | 6132/48050 [09:28<52:24, 13.33it/s]


 13%|████▊                                 | 6134/48050 [09:29<50:53, 13.73it/s]


 13%|████▌                               | 6136/48050 [09:29<1:02:58, 11.09it/s]


 13%|████▌                               | 6138/48050 [09:29<1:30:14,  7.74it/s]


 13%|████▌                               | 6140/48050 [09:29<1:18:48,  8.86it/s]


 13%|████▌                               | 6142/48050 [09:30<1:15:24,  9.26it/s]


 13%|████▌                               | 6144/48050 [09:30<1:20:08,  8.72it/s]


 13%|████▌                               | 6146/48050 [09:30<1:17:28,  9.01it/s]


 13%|████▌                               | 6148/48050 [09:30<1:14:25,  9.38it/s]


 13%|████▌                               | 6150/48050 [09:31<1:13:29,  9.50it/s]


 13%|████▌                               | 6152/48050 [09:31<1:03:24, 11.01it/s]


 13%|████▌                               | 6154/48050 [09:31<1:12:10,  9.68it/s]


 13%

 13%|████▋                               | 6313/48050 [09:47<1:28:26,  7.87it/s]


 13%|████▋                               | 6315/48050 [09:47<1:25:12,  8.16it/s]


 13%|████▋                               | 6317/48050 [09:47<1:29:43,  7.75it/s]


 13%|████▋                               | 6318/48050 [09:48<1:55:45,  6.01it/s]


 13%|████▋                               | 6320/48050 [09:48<1:37:30,  7.13it/s]


 13%|████▋                               | 6321/48050 [09:48<1:35:20,  7.29it/s]


 13%|████▋                               | 6322/48050 [09:48<1:31:52,  7.57it/s]


 13%|████▋                               | 6323/48050 [09:48<1:27:30,  7.95it/s]


 13%|████▋                               | 6325/48050 [09:48<1:21:29,  8.53it/s]


 13%|████▋                               | 6327/48050 [09:48<1:08:46, 10.11it/s]


 13%|████▋                               | 6329/48050 [09:49<1:06:25, 10.47it/s]


 13%|████▋                               | 6331/48050 [09:49<1:05:58, 10.54it/s]


 13%

In [24]:
pd.Series(relations).value_counts()[:50]

NameError: name 'relations' is not defined

Time it took to do it. Whenever doing with GPT model will need to develop prompts but doing through the API have to write code. If want relationships in a specif format, would have to create relationships into a file. But then how does Spacy do that? Is that the same order of time to do that in a GPT model? Can it do it automatically and faster? Is there a much lower bar to make it happen? Even if Spacy is better. 

When we start evaluating downfalls of these models that they don't have memory. The Raven one has infinite memory but looking at GPT based ones we have to use zero shot memory by injecting into data prompts themselves. Are there things that we can do to improve results? Is there an idea around extracting relationship information that even if GPT model does it so now we can inject it inot zero shot lerning to improve how it eva,uates on subsequent topics. Zero shot training. AutoGPT and Jarvis from microsoft is the corrollary to that. 

RE from Spacy and GPT comparison
Article is trying to visualize LLM as it's being trained. Look at this capability. Knowledge of the embedding space. 
Taking HT data, parsing into sections that make sense. Whether that's paragraphs and creating summaries of those sections and using one of the embedding systems to store them in a vector set to use for searching. Now can take prompts from LLM and pass it's concepts and get similaritiues from vector data space and inject those summaries intot he prompt so giving LLM. Doing preprocessing step by taking initial query and before pass it to the LLM getting it's embeddings from that data vector base are relevant and getting articles linked into embeddings. Now prompt will have original question and utilizing info from vector and all of that goes into the prompt. If want the bot to answer a certain way, can use data provided to answer. Will have command of the language from LLM but have relevant information from data set. Have to summarize prompts to have that new vector background info. 

In a HT use case, I want o figure out what places and why they are utilized for trafficking locations? The LLM says: what would I need to do to figure that out: break down into tasks and decides to do web queries, do stat analysis etc. And tasks are autogenerated and at the end it provides a result based on all the tasks it's created. 

Worthwhile getting exposed to all of this. Use the latest and greatest model. LLMs 3.5 and 4. Getting a feel for capacity and capability. 

Looking at trying to utilize this and figure out weakensses and strengths and what can be done to improve those weaknesses. Touch as many tools and things: pine cone for vector databases, autoGPT for task creation. 

Creating pinecone account to create vector database for long-term memory. 

Preprocessing prompt to look for similarities. 